In [1]:
import numpy as np
import pandas as pd
import tweepy as tw
import json

_keys_file = open('tokens.json')
keys = json.load(_keys_file)

### Authenticate

In [2]:
auth = tw.OAuthHandler(keys['api_key'], keys['api_secret'])
api = tw.API(auth, wait_on_rate_limit=True)

### Get Tweets

In [9]:
search_words = "@Tigo_GT -filter:retweets"
date_since = "2022-01-01"

# Collect tweets
tweetsResults = tw.Cursor(
  api.search_tweets,
  q=search_words,
  lang="es",
).items(1500)

In [10]:
tweets = []
for tweet in tweetsResults: 
  tweets.append(tweet)
print("Total Tweets fetched:", len(tweets))

Total Tweets fetched: 1500


In [11]:
# Init dataframe
data = []
columns = ['userName', 'userLocation', 'userDescription', 'isUserVerified', 'date', 'text', 'source']

for tweet in tweets:
  try:text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
  except: pass

  # Adding new data
  data.append([
    tweet.user.name, 
    tweet.user.location,
    tweet.user.description,
    tweet.user.verified,
    tweet.created_at,
    text, 
    tweet.source
  ])

# show the dataframe
df = pd.DataFrame(columns=columns, data=data)

Rate limit reached. Sleeping for: 701


In [12]:
df.head()

,userName,userLocation,userDescription,isUserVerified,date,text,source
0,Lourdes Benavente,Latitud flor y granizo,Ingeniera Ambiental de profesión y ambientalis...,False,2022-09-11 17:17:22+00:00,Aparece el 5G y la red de tigo ya no sirve par...,Twitter for Android
1,Mariela,Guatemala,"Una bad bitch, girl de lo' 90",False,2022-09-11 17:05:36+00:00,@Tigo_GT Entregarte tanto fue mi debilidad\nY ...,Twitter for iPhone
2,Mariela,Guatemala,"Una bad bitch, girl de lo' 90",False,2022-09-11 17:05:24+00:00,@Tigo_GT Entregarte tanto fue mi debilidad\nY ...,Twitter for iPhone
3,Mariela,Guatemala,"Una bad bitch, girl de lo' 90",False,2022-09-11 17:05:00+00:00,@Tigo_GT Entregarte tanto fue mi debilidad\nY ...,Twitter for iPhone
4,Griffith91,,Blueworld,False,2022-09-11 16:30:06+00:00,@Tigo_GT 👑👑👑👑Super Junior 👑👑👑👑🇬🇹🇬🇹🇬🇹🇬🇹💙💙💙💙💙💙💙💙...,Twitter for Android


In [13]:
df.to_csv('train.csv', index=False, encoding='utf-8')